In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import json
from tqdm import tqdm_notebook as tqdm
import matplotlib.patches as patches
import collections
from random import randint
import random
from IPython.display import Image

In [2]:
Image(url= "flip_cube.jpg", width=300, height=300)

In [3]:
colors = {'g': 'green',
          'b': 'blue',
          'y': 'yellow',
          'p': 'purple',
          's': 'grey',
          'o': 'orange',
          'w': 'white',
          'r': 'red'  
        }

In [4]:
diskColors = ["g", "b"]
sideColors = ["y", "p", "s", "o", "w", "r"]

In [5]:
pieces =  it.chain.from_iterable(zip(diskColors,[x,x]) for x in sideColors)  

In [6]:
config = it.permutations(pieces)

In [7]:
start = list(next(config)) 

In [8]:
target = [('g', 'y'),
 ('g', 'p'),
 ('g', 'r'),
 ('g', 'w'),
 ('g', 's'),
 ('g', 'o'),
 ('b', 'y'),
 ('b', 'p'),
 ('b', 'r'),
 ('b', 'w'),
 ('b', 's'),
 ('b', 'o')]

## Operations affecting a given configuration 

In [9]:
def shearRot(cf):
    lst = cf*1
    lst[0],lst[1],lst[2],lst[6],lst[7],lst[8] = lst[8],lst[7],lst[6], lst[2], lst[1], lst[0]
    return lst

In [10]:
def totalRot(n, cf):
    t, b = cf[0:6], cf[6:12]
    t, b = collections.deque(t), collections.deque(b)
    t.rotate(n)
    b.rotate(n)
    return list(t+b)

In [11]:
def topRot(n, cf):
    t, b = cf[0:6], cf[6:12]
    t, b = collections.deque(t), collections.deque(b)
    t.rotate(n)
    b.rotate(0)
    return list(t+b)

In [12]:
def bottRot(n, cf):
    t, b = cf[0:6], cf[6:12]
    t, b = collections.deque(t), collections.deque(b)
    t.rotate(0)
    b.rotate(n)
    return list(t+b)

In [13]:
def sphere(cf):
    "sphere of radius 1 centered at cf"
    sph = []
    tRp, tRm = topRot(1, cf), topRot(-1, cf)
    bRp, bRm = bottRot(1, cf), bottRot(-1, cf)
    sph.append(tRp)
    sph.append(tRm)
    sph.append(bRp)
    sph.append(bRm)
    for n in range(6):
        totalR= totalRot(n, cf)
        sph.append(shearRot(totalR))
    return sph   

In [14]:
def drunkard_walk(n):
    "randon walk of n steps in the configuration space"
    trace = []
    z = start
    for i in range(n):
        nz = random.choice(sphere(z))
        trace.append(nz)
        z = nz
    return trace

In [15]:
def showConfig(cf):
    "graphics representation of the puzzle"
    tt = [a[0] for a in cf][0:6]  
    mt = [a[1] for a in cf][0:6]
    mb = [a[1] for a in cf][6:12]
    bb = [a[0] for a in cf][6:12] 
    
    ax1=plt.subplot(111,aspect='equal')
    ls = tt + mt + mb + bb
    dx, dy = 1/6, 1/4
    pieces = []
    for j in range(24):
        x, cnt = (j%6)/6, j//6+1
        color = colors[ls[j]]
        p = patches.Rectangle(
                    (x,1-cnt/4), dx,dy, edgecolor="white", facecolor=color, fill=True,linewidth=0.5 )
        pieces.append(p)
    
    for p in pieces: ax1.add_patch(p)
    plt.axis('off')
    plt.show()   

In [16]:
def equiv_class(cf):
    "flipping and whole rotations of a given configuration of the  puzzel "
    ec = []
    for n in range(6):
        ec.append(totalRot(n,cf))
    t, b = cf[6:12], cf[0:6]
    for n in range(6):
        ec.append(totalRot(n, t+b))
    return ec

In [17]:
def dist(x,y):
    "lexicographic distance between two points in the configutaion space"
    def dist_help(x,y):
        d = 0
        for n in range(12):
            if x[n] != y[n]: d= d+1
        return d
    EC = equiv_class(y) 
    dt = [dist_help(x,z) for z in EC]
    return min(dt)

In [18]:
def getRid_dup(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num)        
    return final_list 

In [22]:
def next_sphere(S0,S1):
    S = list(it.chain.from_iterable(sphere(x) for x in S1))
    S2 = getRid_dup(S)
    for x in S0: S2.remove(x)   
    return  getRid_dup(S2)

In [23]:
def n_sphere(n, cf):
    "sphere of radius n centered at cf"
    S0 = [cf]
    S1 = sphere(cf)
    k = 1
    while k < n:
        S2 = next_sphere(S0,S1)
        S0 = S1
        S1 = S2
        k = k+1
    return  getRid_dup(S1)

In [24]:
S0 = [start]
S1 = sphere(start)
S2 = next_sphere(S0,S1)
S3 = next_sphere(S1,S2)
S4 = next_sphere(S2,S3)
S5 = next_sphere(S3,S4)
S6 = next_sphere(S4,S5)

In [25]:
T0 = [target]
T1 = sphere(target)
T2 = next_sphere(T0,T1)
T3 = next_sphere(T1,T2)
T4 = next_sphere(T2,T3)
T5 = next_sphere(T3,T4)
T6 = next_sphere(T4,T5) 

In [26]:
(len(S6), len(T6))

(29687, 29687)

## Triangulation

In [48]:
random.seed(7)

In [49]:
mpt = random.choice(T6)
M0 = [mpt]
M1 = sphere(mpt)
M2 = next_sphere(M0,M1)
M3 = next_sphere(M1,M2)
M4 = next_sphere(M2,M3)    
M5 = next_sphere(M3, M4)
WW = list(it.chain.from_iterable([M1,M2,M3,M4,M5]) )  
W = list(it.chain.from_iterable(equiv_class(x)  for x in WW)) 

In [50]:
sum([S6.count(x) for x in W])

8

In [ ]:
showConfig(start)

In [ ]:
showConfig(topRot(start,1))